In [1]:
pip install /Users/qingchuanlyu/Documents/Conferences/ICML-2025/icml2025/code/mlrose

Processing ./mlrose
  Preparing metadata (setup.py) ... done
  Created wheel for mlrose_hiive: filename=mlrose_hiive-2.2.4-py3-none-any.whl size=132417 sha256=543c79d8d002533b2faddba3d0fd2ea8fec5da0aed9b581a972af0ab2003c5a2
  Stored in directory: /private/var/folders/c6/5zg6wy5n121f2pzclkwbzplm0000gn/T/pip-ephem-wheel-cache-6xst1w3b/wheels/77/c5/de/7a63a2c78a12ad335a5091a9209439e12d86616744464a5ae4
Successfully built mlrose_hiive
  Attempting uninstall: mlrose_hiive
    Found existing installation: mlrose_hiive 2.2.4
    Uninstalling mlrose_hiive-2.2.4:
      Successfully uninstalled mlrose_hiive-2.2.4
Note: you may need to restart the kernel to use updated packages.


In [2]:
import itertools
import random
import time
import numpy as np
import mlrose_hiive as mh
import utility as u

In [3]:
class Task:
    def __init__(self, task_id, duration, deadline, profit):
        self.task_id = task_id
        self.duration = duration
        self.deadline = deadline
        self.profit = profit

In [4]:
work_limit = 2
break_time = 1

In [ ]:
""" example
    Task('A', 2, 5, 50),
    Task('B', 3, 8, 70),
    Task('C', 1, 4, 40),
    Task('D', 2, 6, 60),
    Task('E', 4, 10, 100)
"""

In [ ]:
# generate random tasks

In [ ]:
# task size = 3

In [14]:
tasks = [Task('A', 2, 5, 50),
    Task('B', 3, 8, 70),
    Task('C', 1, 4, 40)]

In [ ]:
# evaluate fitness

In [5]:
def evaluate(schedule): #, tasks, break_time, work_limit):  # maximize profit
    time = 0
    fitness = 0
    order = []
    for task_id in schedule:
        task = tasks[task_id]
        #print("current taks is ", task.task_id)
        #print("expected finish time", time + task.duration)
        #print("deadline", task.deadline)
        expected_time = time + task.duration
        if expected_time <= task.deadline:
            #print("finish a task")
            fitness += task.profit
            order.append(task_id)
            if (task.duration >= work_limit) | (expected_time % work_limit == 0 and time != 0):
                #print("take a break")
                order.append('-1')
                time += break_time
            time += task.duration
        #print(order)
    return order, fitness

In [ ]:
# test

In [14]:
evaluate([1, 0, 1])

current taks is  B
expected finish time 3
deadline 8
finish a task
take a break
after a break, the time is  1
[1, '-1']
current taks is  A
expected finish time 6
deadline 5
[1, '-1']
current taks is  B
expected finish time 7
deadline 8
finish a task
take a break
after a break, the time is  5
[1, '-1', 1, '-1']


140

In [ ]:
# verify size = 3

In [18]:
start_time = time.time()
candidate_schedule = list(itertools.product([0, 1, 2], repeat=3))
sch = []
fch = []
for c in candidate_schedule:
    s = list(c)
    order, fitness = evaluate(s)
    sch.append(order)
    fch.append(fitness)
index_max = max(range(len(fch)), key=fch.__getitem__)
print(sch[index_max])
run_time = time.time() - start_time
print(max(fch))
print(run_time)

[2, 1, '-1', 1, '-1']
180
0.0016598701477050781


In [ ]:
# verify size = 7

In [1]:
import importlib
import utility as u

In [6]:
size = 7
importlib.reload(u)
tasks = u.generate_random_tasks(size, 5, 80, 25)

In [9]:
start_time = time.time()
candidate_schedule = list(itertools.product(list(range(7)), repeat=7))
sch = []
fch = []
for c in candidate_schedule:
    s = list(c)
    order, fitness = evaluate(s)
    sch.append(order)
    fch.append(fitness)
index_max = max(range(len(fch)), key=fch.__getitem__)
print(sch[index_max])
run_time = time.time() - start_time
print(max(fch))
print(run_time)

[6, '-1', 6, '-1', 6, '-1', 6, '-1', 6, '-1', 6, '-1', 6, '-1']
161
8.646990299224854


In [ ]:
# size 10

In [25]:
size = 10
#importlib.reload(u)
tasks = u.generate_random_tasks(size, 5, 80, 25)

In [26]:
# # Job Scheduling
problem_ffp = mh.JobSchedulingGenerator().generate(seed=0, size=size, tasks=tasks, break_time=1, work_limit=2)
#state = problem_ffp.get_state()

In [13]:
# GAJ runner class
ga_ffp = mh.GARunner(problem=problem_ffp,
              experiment_name='ffp_ga',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=0,
              iteration_list=2 ** np.arange(12),
              max_attempts=500,
              population_sizes=[1000],
              mutation_rates=[0.08]) # Probability of a mutation at each element of the state vector during reproduction, expressed as a value between 0 and 1.

# the two data frames will contain the results
df_run_stats_ffp, df_run_curves_ffp = ga_ffp.run()

In [16]:
df_run_stats_ffp.head()

,Iteration,Fitness,FEvals,Time,State,Population Size,Mutation Rate,max_iters
0,0,88.0,0,0.030996,"[7, 6, 8, 8, 1, 6, 7, 7, 8, 1]",1000,0.08,2048
1,1,161.0,2,0.205177,"[8, 5, 1, 9, 7, 2, 6, 0, 9, 1]",1000,0.08,2048
2,2,169.0,4,0.298804,"[8, 5, 5, 6, 0, 9, 1, 9, 6, 3]",1000,0.08,2048
3,4,182.0,8,0.734613,"[2, 1, 8, 6, 6, 9, 2, 6, 9, 3]",1000,0.08,2048
4,8,215.0,15,1.690638,"[8, 8, 6, 5, 6, 9, 6, 6, 6, 9]",1000,0.08,2048


In [17]:
df_run_stats_ffp.Fitness.max()

230.0

In [9]:
# GAB-B runner class
ga_ffp = mh.GAJSRunner(problem=problem_ffp,
              experiment_name='ffp_ga',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=0,
              iteration_list=2 ** np.arange(12),
              max_attempts=500,
              population_sizes=[1000],
              mutation_rates=[0.08]) # Probability of a mutation at each element of the state vector during reproduction, expressed as a value between 0 and 1.

# the two data frames will contain the results
df_run_stats_ffp, df_run_curves_ffp = ga_ffp.run()

In [10]:
df_run_curves_ffp['Fitness'].max()

230.0

In [27]:
# SA runner class
sa_ffp = mh.SARunner(problem=problem_ffp,
                     experiment_name='ffp_sa',
                     output_directory=None,  # note: specify an output directory to have results saved to disk
                     seed=0,
                     iteration_list=2 ** np.arange(12),
                     max_attempts=500,
                     temperature_list=[1, 2, 5, 10, 20, 50, 100],
                     decay_list=[mh.ExpDecay])

# the two data frames will contain the results
df_run_stats_ffp, df_run_curves_ffp = sa_ffp.run()

In [28]:
df_run_stats_ffp.Fitness.max()

230.0

In [ ]:
# size 13

In [21]:
size = 13
tasks = u.generate_random_tasks(size, 5, 80, 25)

In [22]:
# # Job Scheduling
problem_ffp = mh.JobSchedulingGenerator().generate(seed=0, size=size, tasks=tasks, break_time=1, work_limit=2)
#state = problem_ffp.get_state()

In [10]:
# GAJ runner class
ga_ffp = mh.GARunner(problem=problem_ffp,
              experiment_name='ffp_ga',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=0,
              iteration_list=2 ** np.arange(12),
              max_attempts=500,
              population_sizes=[1000],
              mutation_rates=[0.08]) # Probability of a mutation at each element of the state vector during reproduction, expressed as a value between 0 and 1.

# the two data frames will contain the results
df_run_stats_ffp, df_run_curves_ffp = ga_ffp.run()

In [11]:
df_run_stats_ffp.Fitness.max()

299.0

In [13]:
# GAB-B runner class
ga_ffp = mh.GAJSRunner(problem=problem_ffp,
              experiment_name='ffp_ga',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=0,
              iteration_list=2 ** np.arange(12),
              max_attempts=500,
              population_sizes=[1000],
              mutation_rates=[0.08]) # Probability of a mutation at each element of the state vector during reproduction, expressed as a value between 0 and 1.

# the two data frames will contain the results
df_run_stats_ffp, df_run_curves_ffp = ga_ffp.run()

In [14]:
df_run_curves_ffp['Fitness'].max()

299.0

In [23]:
# SA runner class
sa_ffp = mh.SARunner(problem=problem_ffp,
                     experiment_name='ffp_sa',
                     output_directory=None,  # note: specify an output directory to have results saved to disk
                     seed=0,
                     iteration_list=2 ** np.arange(12),
                     max_attempts=500,
                     temperature_list=[1, 2, 5, 10, 20, 50, 100],
                     decay_list=[mh.ExpDecay])

# the two data frames will contain the results
df_run_stats_ffp, df_run_curves_ffp = sa_ffp.run()

In [24]:
df_run_stats_ffp.Fitness.max()

299.0

In [12]:
# size 18

In [15]:
size = 18
tasks = u.generate_random_tasks(size, 5, 80, 25)

In [16]:
# # Job Scheduling
problem_ffp = mh.JobSchedulingGenerator().generate(seed=0, size=size, tasks=tasks, break_time=1, work_limit=2)
#state = problem_ffp.get_state()

In [16]:
# GAJ runner class
ga_ffp = mh.GARunner(problem=problem_ffp,
              experiment_name='ffp_ga',
              output_directory=None, # note: specify an output directory to have results saved to disk
              seed=0,
              iteration_list=2 ** np.arange(12),
              max_attempts=500,
              population_sizes=[1000],
              mutation_rates=[0.08]) # Probability of a mutation at each element of the state vector during reproduction, expressed as a value between 0 and 1.

# the two data frames will contain the results
df_run_stats_ffp, df_run_curves_ffp = ga_ffp.run()

In [17]:
df_run_stats_ffp.Fitness.max()

422.0

In [19]:
# SA runner class
sa_ffp = mh.SARunner(problem=problem_ffp,
                     experiment_name='ffp_sa',
                     output_directory=None,  # note: specify an output directory to have results saved to disk
                     seed=0,
                     iteration_list=2 ** np.arange(12),
                     max_attempts=500,
                     temperature_list=[1, 2, 5, 10, 20, 50, 100],
                     decay_list=[mh.ExpDecay])

# the two data frames will contain the results
df_run_stats_ffp, df_run_curves_ffp = sa_ffp.run()

In [20]:
df_run_stats_ffp.Fitness.max()

422.0

In [29]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
